In [4]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt
import cld3
%matplotlib inline

# Clean review data

In [2]:
# Crate data frame
df = pd.read_csv('review_scraper_all.csv')
df.head(3)

,Unnamed: 0,product_page,review_rating,review_content
0,0,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,5.0 out of 5 stars,I like everything about this speaker. the bass...
1,1,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,1.0 out of 5 stars,Se rompió la primera semana y no tuvo ninfun u...
2,2,"Sonos Move - Battery-powered Smart Speaker, Wi...",5.0 out of 5 stars,Received this as a gift and I love it! Easy to...


In [7]:
# Rename the columns
df3 = df.rename(columns={'product_page': 'product_name',
                          'review_content': 'content'})
df3.head(3)

,Unnamed: 0,product_name,review_rating,content
0,0,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,5.0 out of 5 stars,I like everything about this speaker. the bass...
1,1,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,1.0 out of 5 stars,Se rompió la primera semana y no tuvo ninfun u...
2,2,"Sonos Move - Battery-powered Smart Speaker, Wi...",5.0 out of 5 stars,Received this as a gift and I love it! Easy to...


# Language Detection

In [9]:
language = []

for i in range(len(df3)):
  l = cld3.get_language(df3.content[i])[0]
  language.append(l)
  df_lan = pd.DataFrame({'language':language})
  
df_lan

,language
0,en
1,es
2,en
3,en
4,en
...,...
232688,en
232689,es
232690,en
232691,lt


In [32]:
df3_1 = pd.concat([df3, df_lan], axis=1)
df3_1.shape

(232693, 5)

In [33]:
df3_1.to_csv('review_scraper_all_lan.csv')

In [16]:
df3_2 = df3_1[~df3_1.language.isin(['es','fr','it','ja'])]
print(df3_2.shape)

(211232, 5)


In [34]:
df3_2.to_csv('review_scraper_all_cln.csv')

# Merge review data with price data

In [36]:
df_pr = pd.read_csv('price_clusters.csv')
df_pr.head(3)

,Unnamed: 0,product_name,price,y_value,cluster
0,0,UpBright AUX in Cable Audio in Cord Compatible...,3.99,0,3
1,1,UPBRIGHT New USB PC Charging CABE Cord Lead fo...,3.99,0,3
2,2,"125 Bass Speaker Voice Coil, Speaker Coil Repl...",4.49,0,3


In [37]:
df4 = df3_2.merge(df_pr, on='product_name')
df4.head(3)

,Unnamed: 0_x,product_name,review_rating,content,language,Unnamed: 0_y,price,y_value,cluster
0,0,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,5.0 out of 5 stars,I like everything about this speaker. the bass...,en,4202,78.94,0,0
1,3,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,5.0 out of 5 stars,Great,en,4202,78.94,0,0
2,7,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,5.0 out of 5 stars,Love it! At some Wal-Mart stores they have the...,en,4202,78.94,0,0


In [38]:
# Drop useless columns
df5 = df4[['cluster', 'product_name', 'price', 'review_rating', 'content']]
df5.head(3)

,cluster,product_name,price,review_rating,content
0,0,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,78.94,5.0 out of 5 stars,I like everything about this speaker. the bass...
1,0,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,78.94,5.0 out of 5 stars,Great
2,0,JBL FLIP 4 - Waterproof Portable Bluetooth Spe...,78.94,5.0 out of 5 stars,Love it! At some Wal-Mart stores they have the...


In [39]:
# Get the first letter of strings in the rating column
df5['rating'] = df5['review_rating'].astype(str).str[0]
df6 = df5[['cluster', 'product_name', 'price', 'rating', 'content']]
df6.shape

(105575, 5)

In [40]:
# Sorting by price
df8 = df6.sort_values('price', ascending=True)

In [41]:
# Save data frame to csv
df8.to_csv('product_reviews.csv')
df8.head()

,cluster,product_name,price,rating,content
99738,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,5,When ordering I was unsure if the holder would...
99739,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,1,Miss leading add it's not useful\nfor anything
99740,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,5,Very adaptable to all different location mounts.
99741,3,ZOEA Portable Bluetooth Speaker Mount for Golf...,7.99,2,I thought it was a speaker but it's just a lit...
77926,3,"Bluetooth Speaker, HotNCold Vintage Surround S...",9.99,5,"Great product, great service."


# Break into five data frames by cluster

In [42]:
rv_c1 = df8[df8.cluster == df8['cluster'].unique()[0]]
rv_c2 = df8[df8.cluster == df8['cluster'].unique()[1]]
rv_c3 = df8[df8.cluster == df8['cluster'].unique()[2]]
rv_c4 = df8[df8.cluster == df8['cluster'].unique()[3]]
rv_c5 = df8[df8.cluster == df8['cluster'].unique()[4]]

In [43]:
# How many reviews in each cluster
print(rv_c1.shape[0])
print(rv_c2.shape[0])
print(rv_c3.shape[0])
print(rv_c4.shape[0])
print(rv_c5.shape[0])

62942
21906
10286
8258
2183
